In [ ]:
### SEPARAÇÃO DA BASE DE PRATOS DE TREINAMENTO E A DE PREDICÃO ###

### OS PRATOS DA BASE ALLRECIPES QUE DAO MATCH COM A BASE DE PRATOS QUE VEIO DA LISTA DE HARMONIZACAO
### SERAO OS PRATOS USADOS PARA TREINAMENTO

### O RESTANTE SERÁ USADO COMO TESTE DE PREDIÇÃO COM O ALGORITMO PRONTO

In [96]:

import pandas as pd
from fuzzywuzzy import process


In [9]:
# Carregar as bases de dados
pratos_allrecipes_filtrados = pd.read_csv('./datasets/raw/ALLRECIPES_PRATOSxINGREDIENTES_filtrado.csv')
pratos_harmonizacao = pd.read_csv('./datasets/preprocessed/PRATOS.csv')

In [51]:
# separando as receitas que tem na minha base de PRATOS das que nao tem

# Listas de nomes de receitas
allrecipe_names = pratos_allrecipes_filtrados['prato'].tolist()
pratos_harmonizacao_names = pratos_harmonizacao['pratos'].tolist()

# Tolerância de similaridade (0 a 100), onde 100 é uma correspondência exata
similarity_threshold = 90

# Encontrar receitas similares
def find_pratos_similares(recipes, pratos, threshold):
    matches = []
    for recipe in recipes:
        best_match, score = process.extractOne(recipe, pratos)
        if score >= threshold:
            matches.append((recipe, best_match, score))
    return matches

# Executar a comparação
pratos_similares = find_pratos_similares(allrecipe_names, pratos_harmonizacao_names, similarity_threshold)

# Criar dataframes para as novas bases
similarity_df = pd.DataFrame(pratos_similares, columns=['pratos_allrecipes', 'pratos_harmonizacao', 'similarity_score'])

# Identificar receitas que estão na base de pratos e as que não estão
pratos_match = similarity_df['pratos_allrecipes'].unique()
pratos_novasEntradas = pratos_allrecipes_filtrados[~pratos_allrecipes_filtrados['prato'].isin(pratos_match)]
pratos_treinamento = pratos_allrecipes_filtrados[pratos_allrecipes_filtrados['prato'].isin(pratos_match)]



In [53]:
# Fazer o merge da base de treinamento com similarity_df para adicionar a coluna pratos_harmonizacao
pratos_treinamento = pd.merge(pratos_treinamento, similarity_df[['pratos_allrecipes', 'pratos_harmonizacao']], 
                        left_on='prato', right_on='pratos_allrecipes', how='left')

print(pratos_treinamento)


                                                prato  \
0                           caramel apple pie cookies   
1     banana and peanut butter 4 ingredient ice cream   
2                    banana cinnamon chocolate sorbet   
3   flourless peanut butter chocolate chip blender...   
4        abc pudding avocado banana chocolate delight   
..                                                ...   
80                         coffee and donuts tiramisu   
81                            chocolate rum snowballs   
82                               stuffed bell peppers   
83                 crushed pineapple upside down cake   
84                 crushed pineapple upside down cake   

                                         ingredientes  \
0   {'apple', 'frog fish and sea devils', 'sugar',...   
1   {'sugar', 'vanilla', 'ananas', 'peanut butter'...   
2   {'cinnamon', 'vanilla', 'milk powder', 'ananas...   
3   {'honey', 'almond', 'white-flowered gourd', 's...   
4        {'avocado', 'ananas',

In [74]:
# BASE QUE TERÁ TODOS OS MATCHINGS DE PRATOS E INGREDIENTES

df_pratos_ingredientes = pd.concat([pratos_novasEntradas, pratos_treinamento], ignore_index=True)


In [91]:
print(df_pratos_ingredientes)

                                  prato  \
0                    air fryer eggplant   
1            shiitake mushroom stir fry   
2    eggplant caponata sicilian version   
3                     caprese appetizer   
4                     sarahs applesauce   
..                                  ...   
404                     Stuffed peppers   
405                          Strawberry   
406                            Tiramisu   
407                           Chocolate   
408                     Stuffed peppers   

                                          ingredientes  \
0    {'pepper', 'thyme', 'eggplant', 'garlic', 'avo...   
1    {'onion', 'soya sauce', 'pepper', 'mushroom', ...   
2    {'onion', 'plum', 'sugar', 'celery', 'olive', ...   
3    {'pepper', 'olive', 'tomato', 'salt', 'mozzare...   
4              {'cinnamon', 'water', 'sugar', 'apple'}   
..                                                 ...   
404  {'pepper', 'cheddar cheese', 'water', 'corn oi...   
405  {'cheese', 'bu

In [89]:
# remove duplicadas
df_pratos_ingredientes = df_pratos_ingredientes.drop_duplicates()

In [93]:

df_pratos_ingredientes.to_csv('./datasets/preprocessed/PRATOSxINGREDIENTES.csv', index=False)

In [55]:

# Manter apenas as colunas 'prato' (que vem da base harmonizacao), prato_allrecipes e 'ingredientes'
pratos_treinamento = pratos_treinamento.drop(columns=['prato'])

pratos_treinamento = pratos_treinamento.rename(columns={'pratos_harmonizacao': 'prato'})


In [60]:
# remove duplicadas
pratos_treinamento = pratos_treinamento.drop_duplicates()


In [98]:
pratos_treinamento

,ingredientes,pratos_allrecipes,prato
0,"{'apple', 'frog fish and sea devils', 'sugar',...",caramel apple pie cookies,Apple pie
1,"{'sugar', 'vanilla', 'ananas', 'peanut butter'...",banana and peanut butter 4 ingredient ice cream,Ice cream
2,"{'cinnamon', 'vanilla', 'milk powder', 'ananas...",banana cinnamon chocolate sorbet,Chocolate
3,"{'honey', 'almond', 'white-flowered gourd', 's...",flourless peanut butter chocolate chip blender...,Chocolate
4,"{'avocado', 'ananas', 'fatty fish', 'cocoa'}",abc pudding avocado banana chocolate delight,Chocolate
...,...,...,...
78,"{'pepper', 'cheddar cheese', 'water', 'corn oi...",jazzy stuffed peppers,Stuffed peppers
79,"{'cheese', 'butter', 'cream cheese', 'milk', '...",no bake sugar free strawberry cheesecake,Strawberry
80,"{'sugar', 'vanilla', 'milk powder', 'milk', 'c...",coffee and donuts tiramisu,Tiramisu
81,"{'sugar', 'milk powder', 'syrup', 'rum', 'choc...",chocolate rum snowballs,Chocolate


In [62]:
# Salvar as novas bases em arquivos CSV, se necessário
pratos_novasEntradas.to_csv('./datasets/final/PRATOSxINGREDIENTES_novasEntradas.csv', index=False)
pratos_treinamento.to_csv('./datasets/train/PRATOSxINGREDIENTES_treinamento.csv', index=False)

# Salvar o DataFrame de similaridade, se necessário
#similarity_df.to_csv('similarity_df.csv', index=False)